# 2차 라우터

In [ ]:
# routers/rag_handoff_router.py (예시)
RAG_KEYWORDS = ["어떻게", "무엇을", "정책", "혜택", "절차", "방법", "기간", "대상"]
HANDOFF_KEYWORDS = ["불만", "문제", "상담", "접수", "신고", "긴급", "담당자"]

def route_to_rag_or_handoff(state: GraphState) -> str:
    question = state["question"].lower()
    has_rag_keyword = any(kw in question for kw in RAG_KEYWORDS)
    has_handoff_keyword = any(kw in question for kw in HANDOFF_KEYWORDS)

    if has_handoff_keyword:
        return "handoff"  # 인간 담당자 이관
    elif has_rag_keyword:
        return "rag"      # RAG 엔진으로
    else:
        # RAG 키워드 없을 시 기본값: RAG로 처리 (혹은 HANDOFF로 변경 가능)
        return "rag" 

In [ ]:
# routers/rag_handoff_router.py
from typing import Dict
from langgraph.graph import GraphState  # 공유 state 파일 import (팀과 공유)

# (옵션) LLM 설정: LangChain이나 OpenAI API 사용. 초보자라면 더미로 시작.
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-3.5-turbo")

def route_to_rag_or_handoff(state: GraphState) -> str:
    """
    2차 라우터: 질문이 RAG로 처리 가능한지 판단.
    - 반환값: "rag" (RAG 노드로) 또는 "handoff" (담당자 안내 노드로)
    """
    question = state["question"]  # state에서 정제된 질문 읽기
    
    # Level 1: 키워드 기반 판단 (초기 구현 추천)
    handoff_keywords = ["신청", "승인", "상담", "민원", "개인", "내", "나의", "결재"]
    if any(keyword in question for keyword in handoff_keywords):
        print(f"라우팅: RAG 불가능 (키워드: {handoff_keywords}) -> 담당자 안내")
        return "handoff"  # 그래프에서 'handoff' 엣지 이름으로 연결될 값
    
    # Level 2: LLM 기반 판단 (나중에 추가, LLM이 있으면)
    # prompt = f"이 HR 질문이 지식 기반(RAG)으로 답변 가능한가? 아니면 사람 상담이 필요한가? (rag/handoff로 답변): {question}"
    # decision = llm.invoke(prompt).content.strip().lower()
    # if decision == "handoff":
    #     print("라우팅: LLM 판단 -> 담당자 안내")
    #     return "handoff"
    
    # 기본: RAG 가능으로 가정
    print("라우팅: RAG 가능 -> RAG 엔진")
    return "rag"  # 그래프에서 'rag' 엣지 이름으로 연결될 값

# 테스트 코드 (단독 테스트용)
if __name__ == "__main__":
    mock_state = {"question": "내 연봉 인상 신청 어떻게 해?"}
    print(route_to_rag_or_handoff(mock_state))  # 출력: "handoff"